In [1]:
import apiKey
import pinecone
import numpy as np
import pandas as pd
import collections

In [10]:
# create index
 # init
pinecone.init(api_key=apiKey.apiKey, environment="us-west1-gcp")
# create index
index_name = 'dogs'
# no doubles
if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)
# Create data
dimensions = 3
pinecone.create_index(name=index_name, dimension=dimensions, metric="euclidean", shards=1)
# List all existing indices for you API key
print(pinecone.list_indexes())

['dogs']


In [11]:
df = pd.read_csv('model.csv')
df['vectors'] = list(zip(df.x,df.y,df.dist))
df = df[0:999]
df.head()

,cords,emotion,x,y,dist,vectors
0,"(0.49, 0.76)",fear,0.49,0.76,0.576322,"(0.49, 0.76, 0.5763222682136837)"
1,"(0.42, 0.49)",fear,0.42,0.49,0.306565,"(0.42, 0.49, 0.3065653631347322)"
2,"(0.77, 0.48)",fear,0.77,0.48,0.578120,"(0.77, 0.48, 0.5781201270622651)"
3,"(0.13, 0.85)",fear,0.13,0.85,0.622212,"(0.13, 0.85, 0.6222122275716417)"
4,"(0.36, 0.18)",joy,0.36,0.18,0.129225,"(0.36, 0.18, 0.12922501727882832)"


In [12]:
# UPSERT
index = pinecone.Index(index_name=index_name)
# Upsert vectors
index.upsert(vectors=zip(df.emotion, df.vectors))

{'upserted_count': 999}

In [13]:
# query
index.query(
  queries=[[0.49, 0.76, 0.59]],
  top_k=3,
  include_values=True
)

{'results': [{'matches': [{'id': 'surprise',
                           'score': 0.999999285,
                           'values': [0.51, 0.79, 0.612004459]},
                          {'id': 'fear',
                           'score': 0.999942958,
                           'values': [0.49, 0.76, 0.576322258]},
                          {'id': 'joy',
                           'score': 0.999837816,
                           'values': [0.51, 0.76, 0.585112929]}],
              'namespace': ''}]}